## Mars vs Earth Seismology Comparison

In [ ]:
# import necessary files for running code
import numpy as np
import matplotlib.pyplot as plt
from obspy import read_inventory, read
from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime, Stream, read, Trace

### Mars Data

Location

In [ ]:


st1 = read("Mars.mseed")
st2 = read('S0931a_DISP.mseed')
st3 = read('S0931a_VEL.mseed')

channels = ['BHZ']

# stream 
st1.filter(type='highpass', freq=0.01)
st1 = st1.select(component='Z')
print(st1)  

#trace 
tr1 = st1[0]
print(tr1)
print(tr1.stats) 
print(tr1.data)

# stream 
st2.filter(type='highpass', freq=0.1)
st2 = st2.select(component='Z')
print(st2)  

#trace 
tr2 = st2[0]
print(tr2)
print(tr2.stats) 
print(tr2.data)

# stream 
st3.filter(type='highpass', freq=0.1)
st3 = st3.select(component='Z')
print(st3)  

#trace 
tr3 = st3[0]
print(tr3)
print(tr3.stats) 
print(tr3.data)

### Earth Data

Location 